In [204]:
import pandas as pd
import numpy as np
import folium
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup as BS
import re
import warnings
import bokeh
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codecs
from json import JSONDecoder
from functools import partial

from folium import plugins
from folium.plugins import HeatMap
import datetime
import geopy.distance
import mpu
from folium.plugins import MarkerCluster

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
warnings.filterwarnings("ignore")

In [205]:
faults = pd.read_csv('data\\J1939Faults.csv',header=None)


In [206]:
columns1=["FaultId",
"ESS_Id",
"EventTimeStamp",
"eventDescription",
"actionDescription",
"ecuSoftwareVersion",
"ecuSerialNumber",
"ecuModel",
"ecuMake",
"ecuSource",
"spn",
"fmi",
"active",
"activeTransitionCount",
"faultValue",
"EquipmentID",
"MCTNumber",
"Latitude",
"Longitude",
"LocationTimeStamp"] 

In [207]:
faults.columns=columns1
faults.shape

(1085683, 20)

In [208]:

faults.shape

(1085683, 20)

In [209]:
faults.EventTimeStamp = pd.to_datetime(faults.EventTimeStamp, format = '%Y-%m-%d %H:%M:%S')

In [269]:
faults['day'] = faults['EventTimeStamp'].dt.day
faults['month'] = faults['EventTimeStamp'].dt.month
faults['Year'] = faults['EventTimeStamp'].dt.year

In [275]:
pd.value_counts(faults['winter'])

0    152021
1     54657
Name: winter, dtype: int64

In [274]:
faults["winter"]=0
faults.loc[faults['month'].isin([12,1,2]),"winter"]=1

In [210]:
faults = faults.loc[faults['active'] == True,:]

In [211]:
faults['EquipmentID'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
faults.shape

(557358, 20)

In [212]:
faults['EquipmentID']=faults['EquipmentID'].astype(int)

In [213]:
faults['EquipmentID'].isna().sum()

0

In [214]:
faults=faults.loc[~faults['EquipmentID'].isin([105349576,105360462,105411041,105427203]), :]
faults.shape

(557248, 20)

In [215]:
faults.head()

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
6,7,990439,2015-02-21 11:40:52,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1597,105344243,36.902916,-86.436481,2015-02-21 11:41:29.000


In [216]:
faults.loc[faults['ecuMake'].isin(['PCAR', '????R',
                                 '?CAR', '?ACCR', '???R']), 'ecuMake']="PACCR"
faults.loc[faults['ecuMake'].isin(['?MMNS', '??MNS']), 'ecuMake']="CMMNS"
faults.loc[faults['ecuMake']=='?NDWS', 'ecuMake']="BNDWS"

In [217]:
faults=faults.loc[(faults["ecuMake"]=="CMMNS"),:]
#faults=faults.loc[(faults["EquipmentID"]>9999),:]
#faults["EquipmentID"].astype(int)

In [218]:
faults.shape

(206678, 20)

In [219]:
bk=faults.groupby("FaultId").agg({"FaultId": [np.size]})
bk.head()

,FaultId
,size
FaultId,
6,1
8,1
9,1
11,1
17,1


In [220]:
list(bk)

[('FaultId', 'size')]

In [221]:
#bk["FaultId"].sort_values("size").head()

In [222]:
diagnostic = pd.read_csv('data\\VehicleDiagnosticOnboardData.csv',header=None)

In [223]:
diagnostic.head()

,0,1,2,3
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [224]:
columns2=["Id","Name","Value","FaultId"]

diagnostic.columns=columns2

diagnostic["Value"].shape

(11687087,)

In [225]:
diagnostic.head()

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [276]:
merged.isna().sum()

ESS_Id                            0
EventTimeStamp                    0
eventDescription               7891
actionDescription            206678
ecuSoftwareVersion               24
ecuSerialNumber                   0
ecuModel                          0
ecuMake                           0
ecuSource                         0
spn                               0
fmi                               0
active                            0
activeTransitionCount             0
faultValue                   206678
EquipmentID                       0
MCTNumber                         0
Latitude                          0
Longitude                         0
LocationTimeStamp                 0
AcceleratorPedal              48406
BarometricPressure             3511
CruiseControlActive            3828
CruiseControlSetSpeed          3229
DistanceLtd                    3424
EngineCoolantTemperature       3560
EngineLoad                     3738
EngineOilPressure              3476
EngineOilTemperature        

In [226]:
merged=faults.set_index('FaultId').join(diagnostic.pivot(
  index = 'FaultId', 
  columns = 'Name', 
  values = 'Value')).reset_index().set_index('FaultId')

In [227]:
bl=diagnostic.pivot(
  index = 'FaultId', 
  columns = 'Name', 
  values = 'Value')

In [228]:
bl.head()

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,
1,0,14.21,False,66.48672,423178.7,100.4,11,0,96.74375,0,1632.2,43.2,12300.907429328,0,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
merged.shape

(206678, 43)

In [230]:
merged.head()

,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000,48,14.4275,False,64.6226,470381.4,181.4,30,38.28,196.5313,1514.5,9480,44,70349.809963756,4.583399,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
8,990441,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
9,990442,2015-02-21 11:40:22,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,15,True,1,NaN,1429,105356054,38.228796,-84.582500,2015-02-21 11:41:44.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000,NaN,14.1375,False,64.6226,257885.9,181.4,80,34.22,218.975,1382.375,5085.35,42.8,37724.957799834,13.72378,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69
17,990504,2015-02-21 11:45:14,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000,37.2,14.1375,False,66.48672,268575.9,183.2,26,33.64,217.2875,1035.5,5004.75,66,39450.13338542,2.166218,NaN,True,60.8,17407,NaN,NaN,36.38905,3276.75,NaN,1.16


In [231]:
merged["full_derated"]=0
merged["partial_derated"]=0

In [232]:
merged.loc[((merged["spn"]==5246) & (merged["fmi"]==0)),"full_derated"]=1

In [233]:
merged.loc[((merged["spn"]==1569) & (merged["fmi"]==31)),"partial_derated"]=1

In [234]:
regression_data=merged

In [235]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206678 entries, 6 to 1130368
Data columns (total 45 columns):
ESS_Id                       206678 non-null int64
EventTimeStamp               206678 non-null datetime64[ns]
eventDescription             198787 non-null object
actionDescription            0 non-null float64
ecuSoftwareVersion           206654 non-null object
ecuSerialNumber              206678 non-null object
ecuModel                     206678 non-null object
ecuMake                      206678 non-null object
ecuSource                    206678 non-null int64
spn                          206678 non-null int64
fmi                          206678 non-null int64
active                       206678 non-null bool
activeTransitionCount        206678 non-null int64
faultValue                   0 non-null float64
EquipmentID                  206678 non-null int32
MCTNumber                    206678 non-null int64
Latitude                     206678 non-null float64
Longitude   

In [236]:
pd.value_counts(merged.ServiceDistance).to_frame().reset_index().rename(columns={'index':"service_distance","ServiceDistance":"counts"})

,service_distance,counts
0,-10774.576171875,4
1,-23487.83203125,3
2,-21334.779296875,2
3,-13235.2060546875,2
4,-23770.5546875,2
5,-16332.7421875,1
6,-12402.5693359375,1
7,-23313.84765625,1
8,-20554.958984375,1
9,-12638.6904296875,1


In [237]:

warning_derated=merged.loc[((merged["spn"]==1569) & (merged["fmi"]==31)),:]
warning_derated.shape

(4456, 45)

In [238]:
warning_derated=warning_derated.drop_duplicates(subset=['EquipmentID', 'EventTimeStamp'], keep="first")
warning_derated.shape

(4445, 45)

In [239]:
full_derated=merged.loc[((merged["spn"]==5246) & (merged["fmi"]==0)),:]

In [240]:
full_derated.shape

(263, 45)

In [241]:
full_derated=full_derated.drop_duplicates(subset=['EquipmentID', 'EventTimeStamp'], keep="first")
full_derated.shape

(262, 45)

In [242]:
full_derated.head(5)

,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,EngineTimeLtd,FuelLevel,FuelLtd,FuelRate,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,full_derated,partial_derated
FaultId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
46,990931,2015-02-21 12:10:51,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1919,1007751,2015-02-22 19:44:55,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2090,1011009,2015-02-23 05:05:44,NaN,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,NaN,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4645.45,NaN,33470.466902374,NaN,NaN,False,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,1,0
6535,1097942,2015-02-26 22:24:29,NaN,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1419,105355995,37.596805,-85.865555,2015-02-26 22:25:05.000,NaN,14.5,False,64.6226,441699.6,185,10,20.3,198.1625,648.125,9087.95,57.2,69605.769379298,0.6340149,NaN,True,140,22527,NaN,NaN,0,3276.75,NaN,0.58,1,0
6629,1104361,2015-02-27 09:09:56,NaN,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,NaN,1486,105338831,40.534259,-76.431805,2015-02-27 09:10:33.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [243]:
# 193 Aldi Blvd, Mt Juliet, TN – 36.0666667, -86.4347222

# 137 Eady Road – Bell Buckle, TN – 35.5883333, -86.4438888

# 5754 Fish Hatchery Road – Russellville, TN  36.1950, -83.174722

# ["Latitude","Longitude"]

full_derated=full_derated.loc[~((full_derated["Latitude"]==36.0) & (full_derated["Longitude"]==-86.4)),:]
full_derated=full_derated.loc[~((full_derated["Latitude"]==35.5) & (full_derated["Longitude"]==-86.4)),:]
full_derated=full_derated.loc[~((full_derated["Latitude"]==36.1) & (full_derated["Longitude"]==-83.1)),:]

In [244]:
full_derated["ecuMake"].value_counts()

CMMNS    262
Name: ecuMake, dtype: int64

In [245]:
bad_truck=full_derated['EquipmentID'].unique().tolist()

In [246]:
bad_truck


[1395,
 1630,
 1419,
 1486,
 1492,
 1383,
 1396,
 1602,
 1431,
 1795,
 1595,
 1366,
 1399,
 1417,
 1591,
 1490,
 1437,
 1584,
 1692,
 1590,
 1398,
 1418,
 1472,
 1560,
 1375,
 1566,
 1778,
 1659,
 1403,
 1567,
 1373,
 1551,
 1571,
 1443,
 1378,
 1452,
 1488,
 1384,
 1582,
 1501,
 1407,
 1603,
 1477,
 1389,
 1519,
 1757,
 1458,
 1401,
 1467,
 1579,
 1575,
 1601,
 1657,
 1592,
 1563,
 1557,
 1605,
 1581,
 1811,
 1573,
 1594,
 1668,
 1643,
 1561,
 1940,
 1839,
 1552,
 1637,
 1554,
 1686,
 1598,
 1828,
 1848,
 1572,
 1831,
 1661,
 1654,
 1917,
 1812,
 1803,
 1814,
 1556,
 1564,
 1585,
 1663,
 1623,
 1800,
 1732,
 1818,
 2009,
 1946,
 1559,
 1711,
 1600,
 1939,
 1704,
 1808,
 1743,
 1822,
 1739,
 1764,
 1846,
 2007,
 1669,
 1819,
 1780,
 2143,
 1718,
 2111,
 1874,
 1689,
 1932,
 1731,
 1794,
 1978,
 1691,
 1942,
 1928,
 1885,
 1921,
 1860,
 1877,
 306,
 1853,
 1758,
 1919,
 1958,
 1683,
 1847,
 2209,
 1815,
 2005,
 2082,
 1857,
 1698,
 1827,
 1918,
 1751,
 1961,
 1985]

In [247]:

bad_ones=faults.loc[faults['EquipmentID'].isin(bad_truck),:]
bad_ones.shape

(93349, 20)

In [248]:
bad_ones.head()

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
5,6,990431,2015-02-21 11:40:22,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1417,105438630,33.043564,-96.179722,2015-02-21 11:40:59.000
10,11,990459,2015-02-21 11:42:19,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1582,105410980,40.158055,-80.151018,2015-02-21 11:42:55.000
16,17,990504,2015-02-21 11:45:14,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190002*082113134117*07700053*I0*BBZ*,79623056,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1601,105430420,38.279629,-78.935509,2015-02-21 11:45:51.000
24,25,990611,2015-02-21 11:50:50,Low (Severity Low) Engine Coolant Level,NaN,04993120*00190040*082113134117*07700053*I0*BBZ*,79620774,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1590,105365537,35.587731,-86.444351,2015-02-21 11:51:25.000
34,35,990771,2015-02-21 12:01:10,Low (Severity Low) Engine Coolant Level,NaN,04993120*00187677*082113134117*07700053*I0*BBZ*,79619763,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1585,105443412,38.904675,-85.823657,2015-02-21 12:01:53.000


In [249]:
bok=bad_ones.groupby(["EquipmentID"])["EquipmentID"].agg({np.size})
bok=bok.sort_values(["size"])
bok

,size
EquipmentID,
2209,10
2009,11
2082,15
1827,17
2111,20
1940,28
1477,31
1978,32
2005,33


In [250]:
bok['size'].mean()

666.7785714285715

In [251]:
good_ones=faults.loc[~(faults['EquipmentID'].isin(bad_truck)),:]
good_ones.shape

(113329, 20)

In [252]:
iyi=good_ones.groupby(["EquipmentID"])["EquipmentID"].agg({np.size})
iyi=iyi.sort_values(["size"])
iyi

,size
EquipmentID,
108077146,1
2233,1
2197,1
105420594,1
2239,1
105311240,1
105311799,1
2049,1
105311862,1


In [253]:
iyi['size'].mean()

160.52266288951841

In [254]:
bk=full_derated.groupby(["Latitude","Longitude"])["Latitude","Longitude"].agg({np.size})
bk=bk["Latitude"].sort_values(["size"])
bk.tail(50)

size
Latitude  Longitude       
36.037222 -83.445370   1.0
36.123333 -86.398842   1.0
36.179861 -86.297916   1.0
36.191527 -86.412268   1.0
36.194768 -83.174629   1.0
36.194953 -83.174629   1.0
36.195046 -83.175185   1.0
          -83.175092   1.0
36.195138 -83.174814   1.0
36.195185 -83.174814   1.0
36.195231 -83.175370   1.0
          -83.174675   1.0
36.195277 -83.175462   1.0
36.195324 -83.174537   1.0
36.267407 -78.400370   1.0
36.141759 -86.397546   1.0
35.145972 -86.578981   1.0
42.063703 -72.109583   1.0
36.023101 -87.341898   1.0
35.151990 -84.954074   1.0
35.181944 -84.801250   1.0
35.297731 -82.405462   1.0
35.393796 -85.991712   1.0
35.447314 -82.523981   1.0
35.469675 -84.652638   1.0
35.490787 -86.410277   1.0
35.491203 -86.429583   1.0
35.492222 -86.407638   1.0
35.494398 -86.071435   1.0
35.587592 -86.444027   1.0
35.587777 -86.444074   1.0
36.023888 -86.605694   1.0
35.587916 -86.443611   1.0
35.588009 -86.443935   1.0
36.021944 -87.342222   1.0
35.998425 -83.777916   1.0
35.997500 -86.595092   1.0
35.927361 -84.062962   1.0
35.874768 -84.235185   1.0
35.587962 -86.443842   1.0
35.757731 -77.869490   1.0
35.611157 -80.520416   1.0
35.588101 -86.443703   1.0
35.588055 -86.443657   1.0
35.724212 -86.321296   1.0
38.342175 -82.941342   2.0
36.194537 -83.174907   2.0
36.021990 -87.342222   2.0
40.178379 -88.240277   3.0
39.616250 -84.231666   6.0

In [255]:
service = pd.read_excel('data\\Service Fault Codes_1_0_0_167.xlsx')

In [256]:
service.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [257]:

service.groupby(["SPN","J1939 FMI"]).size().reset_index()

,SPN,J1939 FMI,0
0,22,3,1
1,22,4,1
2,27,2,1
3,27,3,1
4,27,4,1
5,27,13,1
6,27,16,1
7,29,2,1
8,38,2,1
9,38,3,1


In [258]:
service["SPN"].value_counts()

524286    765
524287    585
611        86
3464       27
102        22
1761       21
105        21
110        21
7581       21
633        21
641        19
1172       18
190        18
3226       18
3216       18
1127       17
1244       17
3364       17
91         17
157        16
8289       16
2791       16
3465       16
159        15
94         15
629        15
103        15
1075       15
3031       15
3470       15
3251       15
1209       14
84         14
3361       14
1292       14
1176       14
51         14
3227       14
3217       14
4363       14
175        14
1188       13
111        13
632        13
4360       13
4334       13
639        13
4765       13
171        12
3246       12
100        12
1231       11
649        11
108        11
97         11
5848       11
1390       11
3057       11
724        11
5401       11
3242       11
5749       11
3249       11
8322       11
168        10
1300       10
132        10
1301       10
521209     10
1131       10
3265       10
1325  

In [259]:
service["Lamp Color"].value_counts()

Amber          4971
Red             881
None            871
Maintenance     401
Name: Lamp Color, dtype: int64

In [260]:
service["J1939 FMI"].value_counts()

31    2331
3      754
4      730
2      426
16     372
18     267
0      233
15     216
9      211
11     191
5      182
7      179
14     143
17     138
13     133
1      120
12     108
10     104
6       86
19      77
20      61
21      49
8       13
Name: J1939 FMI, dtype: int64

In [261]:
output = service.pivot_table(index='SPN', columns='J1939 FMI', values='Cummins Fault Code')

In [262]:
output.head(5)

J1939 FMI,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,31
SPN,,,,,,,,,,,,,,,,,,,,,,,
22,NaN,NaN,NaN,719.0,729.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,1228.0,2271.0,2272.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2348.0,NaN,NaN,1625.0,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,NaN,NaN,5319.0,5317.0,5318.0,NaN,NaN,NaN,NaN,6342.0,NaN,NaN,NaN,6343.0,NaN,NaN,NaN,NaN,1439.0,5328.0,NaN,NaN,NaN
46,NaN,NaN,218.0,216.0,217.0,774.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.0,776.0,NaN,NaN,NaN,NaN


In [263]:
lat=38
long=-84

map=folium.Map(location=[lat,long],zoom_start=2)

map2=folium.Map(location=[lat,long],zoom_start=2)
map3=folium.Map(location=[lat,long],zoom_start=2)

In [264]:
for row in full_derated.iterrows():
    row_values = row[1]
    location = [row_values['Latitude'], row_values['Longitude']]
    marker = folium.CircleMarker(location=location,radius=1)
    marker.add_to(map)
map

In [265]:
truck_no_idea=[105360462,105411041,105427203,105349576]

In [266]:
truck_no_idea=faults.loc[(faults['MCTNumber'].isin(truck_no_idea)),:]
truck_no_idea.shape

(3549, 20)

In [267]:
truck_no_idea

,FaultId,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
2060,2061,1010492,2015-02-23 03:59:59,High (Severity Medium) Engine Exhaust Gas Reci...,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,3058,16,True,1,NaN,1386,105349576,35.713703,-85.723703,2015-02-23 04:00:35.000
3812,3813,1042537,2015-02-24 09:26:38,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,38.180462,-84.805648,2015-02-24 09:27:14.000
5744,5745,1070967,2015-02-25 14:07:35,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,32.518055,-90.093935,2015-02-25 14:08:10.000
5806,5807,1071852,2015-02-25 14:43:53,NaN,NaN,04993120*00028586*040213150018*07700044*I0*BBZ*,79464956,6X1u10D1500000000,CMMNS,0,5397,31,True,1,NaN,1386,105349576,36.829675,-81.503379,2015-02-25 14:44:30.000
5954,5955,1073740,2015-02-25 16:09:21,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,31.826805,-89.399768,2015-02-25 16:09:57.000
6075,6076,1074185,2015-02-25 17:39:31,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,31.836250,-89.464074,2015-02-25 17:40:07.000
6123,6124,1075099,2015-02-25 17:42:02,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,31.836342,-89.464212,2015-02-25 18:20:07.000
6150,6151,1075685,2015-02-25 18:27:57,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,31.978287,-89.288703,2015-02-25 19:00:08.000
38716,39828,2192222,2015-05-01 06:32:27,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,39.941620,-82.534259,2015-05-01 06:33:03.000
40720,41832,2226029,2015-05-03 20:44:53,Low (Severity Low) Engine Coolant Level,NaN,04993120*00174945*060712145341*07700027*I0*BBZ*,79621367,6X1u10D1500000000,CMMNS,0,111,17,True,1,NaN,1598,105411041,36.498472,-84.467546,2015-05-03 20:45:30.000


In [268]:
pd.crosstab(truck_no_idea["EquipmentID"], truck_no_idea["MCTNumber"], dropna=True)

MCTNumber,105349576,105360462,105411041
EquipmentID,,,
1386,2,0,0
1576,43,0,0
1598,0,0,254
1696,0,204,0
1809,0,0,3046
